In [78]:
!pip install mysql-connector-python


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
import mysql.connector

In [80]:
conn=mysql.connector.connect(
    host="localhost",
    user="root",
    password="177013vV$"
)
print(conn)

In [81]:
cursor=conn.cursor()
#Create a function in Python to create the tables (students, courses, enrollments) in school_db.
# cursor.execute("CREATE DATABASE school_db")
cursor.execute("USE school_db")
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    student_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    age INT,
    grade VARCHAR(10)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS courses (
    course_id INT AUTO_INCREMENT PRIMARY KEY,
    course_name VARCHAR(100),
    instructor VARCHAR(100)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS enrollments (
    enroll_id INT AUTO_INCREMENT PRIMARY KEY,
    student_id INT,
    course_id INT,
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
)
""")

print("Tables created successfully.")

Tables created successfully.


In [82]:
#1.  Write a Python program to connect to MySQL and print the list of all available databases.

cursor.execute("SHOW DATABASES")

print("Available Databases:")
for db in cursor:
    print(db[0])


Available Databases:
games_db
information_schema
mysql
performance_schema
sakila
school_db
sys
world


In [83]:
import pandas as pd
students_df = pd.read_csv("./students.csv")
courses_df = pd.read_csv("./courses.csv")
enrollments_df=pd.read_csv("./enrollments.csv")

In [84]:
#3. Write a Python program to insert multiple students and courses into the respective tables using executemany().


student_sql = "INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)"
students = list(students_df[['name', 'age', 'grade']].itertuples(index=False, name=None))
cursor.executemany(student_sql, students)
print(cursor.rowcount, "students inserted.")

course_sql = "INSERT INTO courses (course_name, instructor) VALUES (%s, %s)"
courses = list(courses_df[['course_name', 'instructor']].itertuples(index=False, name=None))
cursor.executemany(course_sql, courses)
print(cursor.rowcount, "courses inserted.")

100 students inserted.
10 courses inserted.


In [85]:
#4.  Fetch and print all rows from the students table in a tabular format.

cursor.execute("SELECT * FROM students")
rows = cursor.fetchall()

# get column names
columns = [desc[0] for desc in cursor.description]

# create DataFrame for nice tabular output
df = pd.DataFrame(rows, columns=columns)

print("Students Table:")
print(df.to_string(index=False))

Students Table:
 student_id       name  age grade                   email
          2       Uma2   23     A     uma2108@example.com
          3     Wendy3   22     B   wendy3329@example.com
          4    Xander4   22     A  xander4870@example.com
          5    Victor5   19     A  victor5418@example.com
          6       Ian6   19     D     ian6677@example.com
          7       Leo7   22     A     leo7695@example.com
          8      Paul8   24     B    paul8840@example.com
          9     Wendy9   23     B   wendy9128@example.com
         10      Leo10   23     A    leo10128@example.com
         11      Uma11   18     C    uma11735@example.com
         12     Rita12   18     B   rita12459@example.com
         13     Yara13   22     B   yara13123@example.com
         14      Bob14   19     C    bob14245@example.com
         15   Quincy15   24     A  quincy1558@example.com
         16     Tina16   19     A   tina16919@example.com
         17    Alice17   25     B  alice17910@example.co

In [86]:
#5.  Write a program to update the grade of a student (given student_id) via a Python input prompt.

sid = input("Student ID: ")
grade = input("New Grade: ")

cursor.execute("UPDATE students SET grade=%s WHERE student_id=%s", (grade, sid))

print("Updated." if cursor.rowcount else "Student not found.")

Updated.


In [87]:
#6.  Delete a student from the students table using Python input for student_id.

sid = input("Enter Student ID to delete: ")

cursor.execute("DELETE FROM students WHERE student_id=%s", (sid,))

print("Deleted." if cursor.rowcount else "Student not found.")


IntegrityError: 1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails (`school_db`.`enrollments`, CONSTRAINT `enrollments_ibfk_1` FOREIGN KEY (`student_id`) REFERENCES `students` (`student_id`))

In [ ]:
#7.  Write a Python script to insert enrollment records linking students with their courses, ensuring foreign key integrity.

enrollments_df['student_id'] = enrollments_df['student_id'].astype(int)
enrollments_df['course_id'] = enrollments_df['course_id'].astype(int)

# fetch valid student_ids and course_ids
cursor.execute("SELECT student_id FROM students")
valid_students = {row[0] for row in cursor.fetchall()}

cursor.execute("SELECT course_id FROM courses")
valid_courses = {row[0] for row in cursor.fetchall()}

# filter only valid enrollments
valid_enrollments = [
    (row.student_id, row.course_id)
    for row in enrollments_df.itertuples(index=False)
    if row.student_id in valid_students and row.course_id in valid_courses
]

invalid_enrollments = [
    (row.student_id, row.course_id)
    for row in enrollments_df.itertuples(index=False)
    if row.student_id not in valid_students or row.course_id not in valid_courses
]

print(f"{len(valid_enrollments)} valid enrollments to insert.")
print(f"{len(invalid_enrollments)} invalid enrollments skipped:", invalid_enrollments[:10])

# insert valid enrollments
if valid_enrollments:
    sql = "INSERT INTO enrollments (student_id, course_id) VALUES (%s, %s)"
    cursor.executemany(sql, valid_enrollments)
    conn.commit()
    print("Inserted successfully.")

198 valid enrollments to insert.
1 invalid enrollments skipped: [(1, 10)]
Inserted successfully.


In [ ]:
#debug check
cursor.execute("SELECT * FROM enrollments")
print(cursor.fetchall())


[(200, 2, 6), (201, 3, 8), (202, 4, 10), (203, 5, 10), (204, 5, 3), (205, 5, 6), (206, 6, 7), (207, 6, 4), (208, 6, 5), (209, 7, 1), (210, 8, 9), (211, 9, 9), (212, 10, 7), (213, 10, 3), (214, 10, 1), (215, 11, 3), (216, 11, 6), (217, 11, 8), (218, 12, 10), (219, 13, 2), (220, 13, 9), (221, 13, 10), (222, 14, 3), (223, 14, 1), (224, 15, 8), (225, 15, 5), (226, 15, 2), (227, 16, 6), (228, 16, 4), (229, 16, 1), (230, 17, 6), (231, 17, 1), (232, 17, 7), (233, 18, 3), (234, 19, 9), (235, 19, 1), (236, 20, 8), (237, 20, 4), (238, 20, 10), (239, 21, 7), (240, 21, 6), (241, 22, 3), (242, 22, 10), (243, 22, 9), (244, 23, 5), (245, 24, 7), (246, 24, 8), (247, 25, 9), (248, 26, 9), (249, 26, 8), (250, 26, 1), (251, 27, 6), (252, 28, 3), (253, 28, 1), (254, 29, 5), (255, 29, 3), (256, 30, 3), (257, 30, 6), (258, 30, 1), (259, 31, 4), (260, 31, 9), (261, 31, 3), (262, 32, 4), (263, 32, 9), (264, 33, 5), (265, 33, 4), (266, 34, 3), (267, 34, 8), (268, 35, 7), (269, 36, 6), (270, 36, 4), (271, 37, 2

In [ ]:
#8.  Fetch all students enrolled in a particular course (given course_id as input).

cid = input("Enter Course ID: ")

# fetch students with course name
query = """
SELECT c.course_name, s.student_id, s.name, s.age, s.grade
FROM courses c
JOIN enrollments e ON c.course_id = e.course_id
JOIN students s ON s.student_id = e.student_id
WHERE c.course_id = %s
"""
cursor.execute(query, (cid,))
rows = cursor.fetchall()

if rows:
    print(f"\nStudents enrolled in Course {cid} - {rows[0][0]}:")
    for r in rows:
        print(f"ID: {r[1]}, Name: {r[2]}, Age: {r[3]}, Grade: {r[4]}")
    print(f"\nTotal students enrolled: {len(rows)}")
else:
    print("No students found for that course.")



Students enrolled in Course 10 - Statistics:
ID: 4, Name: Xander4, Age: 22, Grade: A
ID: 5, Name: Victor5, Age: 19, Grade: A
ID: 12, Name: Rita12, Age: 18, Grade: B
ID: 13, Name: Yara13, Age: 22, Grade: B
ID: 20, Name: Victor20, Age: 19, Grade: D
ID: 22, Name: Victor22, Age: 18, Grade: A
ID: 41, Name: Ian41, Age: 24, Grade: D
ID: 42, Name: Quincy42, Age: 24, Grade: C
ID: 45, Name: Xander45, Age: 23, Grade: C
ID: 46, Name: Zane46, Age: 25, Grade: C
ID: 48, Name: Leo48, Age: 22, Grade: B
ID: 55, Name: Yara55, Age: 22, Grade: A
ID: 59, Name: Oscar59, Age: 21, Grade: B
ID: 70, Name: Jack70, Age: 24, Grade: A
ID: 73, Name: Grace73, Age: 19, Grade: A
ID: 78, Name: Ian78, Age: 22, Grade: D
ID: 87, Name: Sam87, Age: 19, Grade: C
ID: 94, Name: Eva94, Age: 24, Grade: C
ID: 4, Name: Xander4, Age: 22, Grade: A
ID: 5, Name: Victor5, Age: 19, Grade: A
ID: 12, Name: Rita12, Age: 18, Grade: B
ID: 13, Name: Yara13, Age: 22, Grade: B
ID: 20, Name: Victor20, Age: 19, Grade: D
ID: 22, Name: Victor22, Age

In [ ]:
#9. Write a program to display students who are not enrolled in any course

query = """
SELECT s.student_id, s.name, s.age, s.grade
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
WHERE e.course_id IS NULL
"""
cursor.execute(query)
rows = cursor.fetchall()

if rows:
    print("Students not enrolled in any course:")
    for r in rows:
        print(f"ID: {r[0]}, Name: {r[1]}, Age: {r[2]}, Grade: {r[3]}")
else:
    print("All students are enrolled in at least one course.")

Students not enrolled in any course:
ID: 101, Name: Zane1, Age: 18, Grade: C
ID: 102, Name: Uma2, Age: 23, Grade: A
ID: 103, Name: Wendy3, Age: 22, Grade: B
ID: 104, Name: Xander4, Age: 22, Grade: A
ID: 105, Name: Victor5, Age: 19, Grade: A
ID: 106, Name: Ian6, Age: 19, Grade: D
ID: 107, Name: Leo7, Age: 22, Grade: A
ID: 108, Name: Paul8, Age: 24, Grade: B
ID: 109, Name: Wendy9, Age: 23, Grade: B
ID: 110, Name: Leo10, Age: 23, Grade: A
ID: 111, Name: Uma11, Age: 18, Grade: C
ID: 112, Name: Rita12, Age: 18, Grade: B
ID: 113, Name: Yara13, Age: 22, Grade: B
ID: 114, Name: Bob14, Age: 19, Grade: C
ID: 115, Name: Quincy15, Age: 24, Grade: A
ID: 116, Name: Tina16, Age: 19, Grade: A
ID: 117, Name: Alice17, Age: 25, Grade: B
ID: 118, Name: Sam18, Age: 20, Grade: B
ID: 119, Name: Paul19, Age: 23, Grade: A
ID: 120, Name: Victor20, Age: 19, Grade: D
ID: 121, Name: Frank21, Age: 23, Grade: A
ID: 122, Name: Victor22, Age: 18, Grade: A
ID: 123, Name: Karen23, Age: 25, Grade: A
ID: 124, Name: Grace2

In [ ]:
#10.  Perform an INNER JOIN between students and courses using Python to show student names along with the courses they are enrolled in.

query = """
SELECT s.name AS student_name, c.course_name
FROM students s
INNER JOIN enrollments e ON s.student_id = e.student_id
INNER JOIN courses c ON e.course_id = c.course_id
ORDER BY s.name
"""
cursor.execute(query)
rows = cursor.fetchall()

if rows:
    print("Students and their enrolled courses:")
    for r in rows:
        print(f"Student: {r[0]}, Course: {r[1]}")
else:
    print("No enrollments found.")

Students and their enrolled courses:
Student: Alice17, Course: Economics
Student: Alice17, Course: Mathematics
Student: Alice17, Course: Economics
Student: Alice17, Course: Computer Science
Student: Alice17, Course: Mathematics
Student: Alice17, Course: Economics
Student: Alice17, Course: Economics
Student: Alice17, Course: Mathematics
Student: Alice17, Course: Computer Science
Student: Alice17, Course: Computer Science
Student: Alice17, Course: Economics
Student: Alice17, Course: Mathematics
Student: Alice17, Course: Computer Science
Student: Alice17, Course: Mathematics
Student: Alice17, Course: Economics
Student: Alice17, Course: Mathematics
Student: Alice17, Course: Computer Science
Student: Alice17, Course: Computer Science
Student: Alice17, Course: Computer Science
Student: Alice17, Course: Mathematics
Student: Alice17, Course: Economics
Student: Alice86, Course: Chemistry
Student: Alice86, Course: Chemistry
Student: Alice86, Course: Chemistry
Student: Alice86, Course: Chemistry


In [ ]:
#11. Write a Python function to search for a student by name (partial matches allowed using LIKE).

def search_student(cursor, name_part):
    query = "SELECT student_id, name, age, grade FROM students WHERE name LIKE %s"
    cursor.execute(query, ('%' + name_part + '%',))
    rows = cursor.fetchall()
    return rows

results = search_student(cursor, "Alice17")
for r in results:
    print(f"ID: {r[0]}, Name: {r[1]}, Age: {r[2]}, Grade: {r[3]}")

ID: 17, Name: Alice17, Age: 25, Grade: B
ID: 117, Name: Alice17, Age: 25, Grade: B
ID: 217, Name: Alice17, Age: 25, Grade: B
ID: 318, Name: Alice17, Age: 25, Grade: B


In [88]:
#12.  Add a new column email to the students table using Python and update it
'''
import numpy as np

# 1. Add email column
cursor.execute("ALTER TABLE students ADD COLUMN email VARCHAR(100)")

# 2. Generate random emails
cursor.execute("SELECT student_id, name FROM students")
students = cursor.fetchall()

emails = [f"{name.lower().replace(' ', '')}{np.random.randint(1,1000)}@example.com" 
          for _, name in students]

# 3. Update table with generated emails
for (student_id, _), email in zip(students, emails):
    cursor.execute("UPDATE students SET email=%s WHERE student_id=%s", (email, student_id))

print("Email column added and updated with random values.")'''

'\nimport numpy as np\n\n# 1. Add email column\ncursor.execute("ALTER TABLE students ADD COLUMN email VARCHAR(100)")\n\n# 2. Generate random emails\ncursor.execute("SELECT student_id, name FROM students")\nstudents = cursor.fetchall()\n\nemails = [f"{name.lower().replace(\' \', \'\')}{np.random.randint(1,1000)}@example.com" \n          for _, name in students]\n\n# 3. Update table with generated emails\nfor (student_id, _), email in zip(students, emails):\n    cursor.execute("UPDATE students SET email=%s WHERE student_id=%s", (email, student_id))\n\nprint("Email column added and updated with random values.")'

In [89]:
#13.  Create a program to count how many students are enrolled per course and display results in descending order.

query = """
SELECT c.course_name, COUNT(e.student_id) AS student_count
FROM courses c
LEFT JOIN enrollments e ON c.course_id = e.course_id
GROUP BY c.course_name
ORDER BY student_count DESC
"""
cursor.execute(query)
rows = cursor.fetchall()

print("Students enrolled per course:")
for r in rows:
    print(f"Course: {r[0]}, Students Enrolled: {r[1]}")

Students enrolled per course:
Course: Chemistry, Students Enrolled: 196
Course: Philosophy, Students Enrolled: 154
Course: Mathematics, Students Enrolled: 147
Course: History, Students Enrolled: 147
Course: Computer Science, Students Enrolled: 147
Course: Economics, Students Enrolled: 140
Course: Statistics, Students Enrolled: 127
Course: Biology, Students Enrolled: 126
Course: Physics, Students Enrolled: 112
Course: English Literature, Students Enrolled: 91


In [90]:
#14.  Write a Python script that accepts a course name as input and returns all enrolled students with their grades.

course_name = input("Enter course name: ")

query = """
SELECT s.student_id, s.name, s.grade
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE c.course_name = %s
"""
cursor.execute(query, (course_name,))
rows = cursor.fetchall()

if rows:
    print(f"Students enrolled in {course_name}:")
    for r in rows:
        print(f"ID: {r[0]}, Name: {r[1]}, Grade: {r[2]}")
else:
    print(f"No students found for the course '{course_name}'.")

Students enrolled in Physics:
ID: 13, Name: Yara13, Grade: B
ID: 15, Name: Quincy15, Grade: A
ID: 37, Name: Ian37, Grade: D
ID: 43, Name: Quincy43, Grade: D
ID: 47, Name: Eva47, Grade: D
ID: 52, Name: Hannah52, Grade: A
ID: 53, Name: Bob53, Grade: C
ID: 57, Name: Nina57, Grade: B
ID: 60, Name: Yara60, Grade: B
ID: 75, Name: Quincy75, Grade: D
ID: 78, Name: Ian78, Grade: D
ID: 83, Name: Leo83, Grade: C
ID: 90, Name: Zane90, Grade: D
ID: 91, Name: Tina91, Grade: D
ID: 96, Name: Ian96, Grade: C
ID: 99, Name: Bob99, Grade: C
ID: 13, Name: Yara13, Grade: B
ID: 15, Name: Quincy15, Grade: A
ID: 37, Name: Ian37, Grade: D
ID: 43, Name: Quincy43, Grade: D
ID: 47, Name: Eva47, Grade: D
ID: 52, Name: Hannah52, Grade: A
ID: 53, Name: Bob53, Grade: C
ID: 57, Name: Nina57, Grade: B
ID: 60, Name: Yara60, Grade: B
ID: 75, Name: Quincy75, Grade: D
ID: 78, Name: Ian78, Grade: D
ID: 83, Name: Leo83, Grade: C
ID: 90, Name: Zane90, Grade: D
ID: 91, Name: Tina91, Grade: D
ID: 96, Name: Ian96, Grade: C
ID: 99

In [91]:
#15. Export the contents of the students table into a CSV file using Python.

cursor.execute("SELECT * FROM students")
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# create DataFrame and export to CSV
df = pd.DataFrame(rows, columns=columns)
df.to_csv("students_export.csv", index=False)

print("Students table exported to students_export.csv")

Students table exported to students_export.csv


In [92]:
#16.  Write a Python program to import student data from a CSV file into the students table.

# prepare data (ignore student_id if AUTO_INCREMENT)
students = list(students_df[['name', 'age', 'grade']].itertuples(index=False, name=None))

# insert into students table
sql = "INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)"
cursor.executemany(sql, students)
conn.commit()
print(cursor.rowcount, "students imported from CSV.")

100 students imported from CSV.


In [93]:
#17.  Create a stored procedure in MySQL (e.g., GetStudentCourses(student_id)) and call it from Python.

student_id = int(input("Enter Student ID: "))

# call stored procedure
cursor.callproc("GetStudentCourses", [student_id])

# fetch results
for result in cursor.stored_results():
    rows = result.fetchall()
    if rows:
        print(f"Courses for student {student_id}:")
        for r in rows:
            print(f"Student: {r[0]},  Course: {r[1]}")
    else:
        print(f"No courses found for student {student_id}.")



Courses for student 11:
Student: Uma11,  Course: Chemistry
Student: Uma11,  Course: Economics
Student: Uma11,  Course: English Literature
Student: Uma11,  Course: Chemistry
Student: Uma11,  Course: Economics
Student: Uma11,  Course: English Literature
Student: Uma11,  Course: Chemistry
Student: Uma11,  Course: Economics
Student: Uma11,  Course: English Literature
Student: Uma11,  Course: Chemistry
Student: Uma11,  Course: Economics
Student: Uma11,  Course: English Literature
Student: Uma11,  Course: Chemistry
Student: Uma11,  Course: Economics
Student: Uma11,  Course: English Literature
Student: Uma11,  Course: Chemistry
Student: Uma11,  Course: Economics
Student: Uma11,  Course: English Literature
Student: Uma11,  Course: Chemistry
Student: Uma11,  Course: Economics
Student: Uma11,  Course: English Literature


C:\Users\Sravanthi\AppData\Local\Temp\ipykernel_23176\1134019567.py:9: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


In [94]:
#18. Handle exceptions in Python to gracefully catch duplicate entries (e.g. inserting a student with the same ID). 

from mysql.connector import IntegrityError

try:
    sql = "INSERT INTO students (student_id, name, age, grade, email) VALUES (%s, %s, %s, %s, %s)"
    val = (1, "Alice17", 20, "A", "alice@example.com")  # duplicate ID example
    cursor.execute(sql, val)
    conn.commit()
    print("Student inserted successfully.")

except IntegrityError as e:
    if e.errno == 1062:  # Duplicate entry error code
        print("Error: Duplicate entry! A student with this ID already exists.")
    else:
        print(f"MySQL IntegrityError: {e}")

Student inserted successfully.


In [95]:
#19.  

try:
    # start transaction
    conn.start_transaction()

    # 1. Insert a new student
    sql_student = "INSERT INTO students (name, age, grade, email) VALUES (%s, %s, %s, %s)"
    student_data = ("VASISTA101", 22, "A", "VASISTA@outlook.com")
    cursor.execute(sql_student, student_data)
    student_id = cursor.lastrowid  # get new student_id

    # 2. Enroll student in a course (intentionally using wrong course_id to demo rollback)
    sql_enroll = "INSERT INTO enrollments (student_id, course_id) VALUES (%s, %s)"
    enroll_data = (student_id, 10)  # 9999 = non-existing course_id (will fail) ALR TRIED
    cursor.execute(sql_enroll, enroll_data)

    # commit only if both succeed
    print("Transaction successful: student inserted and enrolled.")

except mysql.connector.Error as e:
    # rollback on any error
    conn.rollback()
    print("Transaction failed, rolled back.")
    print("Error:", e)

Transaction successful: student inserted and enrolled.


In [96]:
#20 Close the database connection safely in Python, ensuring both cursor and connection are properly closed.

try:
    # your SQL queries here
    conn.commit()   # commit all changes
    print("Changes committed successfully.")

except Exception as e:
    conn.rollback()  # rollback on error
    print("Error occurred, rolled back:", e)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed safely.")


Changes committed successfully.
Database connection closed safely.
